In [31]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# MPC import
import numpy as np
import matplotlib.pyplot as plt
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [33]:
Ts = 0.05
sim_time = 7.0
H = 5.0
x0 = np.array([             # initial state for testing (12 states)
    0, 0, 0,                # angular velocities wx, wy, wz
    0, 0, 0,                # Euler angles α, β, γ
    0, 0, 0,                # velocities vx, vy, vz 
    0, 0, 0                 # position x, y, z
])  # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)



# Terminal set plots 

Xf_x = mpc.mpc_x.terminal_set
Xf_y = mpc.mpc_y.terminal_set
Xf_z = mpc.mpc_z.terminal_set
Xf_r = mpc.mpc_roll.terminal_set

proj_x = Xf_x.projection([1, 2])  # (Δβ, Δv_x)
proj_y = Xf_y.projection([1, 2])  # (Δα, Δv_y)

lb_z, ub_z = Xf_z.outer_box       # 1D interval for Δv_z

fig = plt.figure(figsize=(12, 9))

ax1 = fig.add_subplot(2, 2, 1)
proj_x.plot(ax=ax1)
ax1.set_title("Terminal set: x-subsystem (Δβ vs Δv_x)")
ax1.set_xlabel(r"$\Delta \beta$ [rad]")
ax1.set_ylabel(r"$\Delta v_x$ [m/s]")
ax1.grid(True)

ax2 = fig.add_subplot(2, 2, 2)
proj_y.plot(ax=ax2)
ax2.set_title("Terminal set: y-subsystem (Δα vs Δv_y)")
ax2.set_xlabel(r"$\Delta \alpha$ [rad]")
ax2.set_ylabel(r"$\Delta v_y$ [m/s]")
ax2.grid(True)

ax3 = fig.add_subplot(2, 2, 3)
# 1D terminal set: plot interval on a line
ax3.hlines(0, lb_z[0], ub_z[0])
ax3.plot([lb_z[0], ub_z[0]], [0, 0], marker="o", linestyle="")
ax3.set_title(f"Terminal set: z-subsystem (Δv_z interval) = [{lb_z[0]:.3f}, {ub_z[0]:.3f}]")
ax3.set_xlabel(r"$\Delta v_z$ [m/s]")
ax3.set_yticks([])
ax3.grid(True)

ax4 = fig.add_subplot(2, 2, 4)
Xf_r.plot(ax=ax4)
ax4.set_title("Terminal set: roll subsystem (Δω_z vs Δγ)")
ax4.set_xlabel(r"$\Delta \omega_z$")
ax4.set_ylabel(r"$\Delta \gamma$ [rad]")
ax4.grid(True)

plt.tight_layout()
plt.show()


t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)


vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);


ValueError: not enough values to unpack (expected 3, got 2)